In [1]:
#import a sparksession
from pyspark.sql import SparkSession

In [2]:
#create a sparksession and store in spark variable
spark=SparkSession.builder \
.appName("test") \
.getOrCreate()

In [3]:
#create a df with 10 numbers
number=spark.range(10).toDF("numbers")
number.show()

+-------+
|numbers|
+-------+
|      0|
|      1|
|      2|
|      3|
|      4|
|      5|
|      6|
|      7|
|      8|
|      9|
+-------+



In [4]:
#from df take only divisibile_by_2 numbers and store in divisibile_by_2 variable
divisible_by_2=number.where("numbers%2=0")
divisible_by_2.show()

+-------+
|numbers|
+-------+
|      0|
|      2|
|      4|
|      6|
|      8|
+-------+



In [5]:
#from df take only divisibile_by_3 numbers and store in divisibile_by_3 variable, do collect,take,show on divisibile_by_3 variable
divisible_by_3=number.where("numbers%3=0")
# divisible_by_3.show()
# divisible_by_3.count()
divisible_by_3.collect()
divisible_by_3.take(5)

[Row(numbers=0), Row(numbers=3), Row(numbers=6), Row(numbers=9)]

In [6]:
#load the csv file and store it in csv_load variable and show the output
csv_load=spark.read.option("header","true").option("inferSchema","true").csv("2015-summary.csv")
csv_load.show()
csv_load.take(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [7]:
#load the csv file and store it in csv_load variable and sort the data with count column and show the output
csv_load=spark.read.option("header","true").option("inferSchema","true").csv("2015-summary.csv")
sort_count=csv_load.sort("count")
sort_count.show()
sort_count.sort("count").explain()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [8]:
#load the csv file and store it in csv_load variable and create a temporary table "sql_csv_table"
csv_load=spark.read.option("header","true").option("inferSchema","true").csv("2015-summary.csv")
csv_load.createOrReplaceTempView("sql_csv_table")

In [9]:
#load the csv file and store it in csv_load variable and create a temporary table "sql_csv_table",
#write a query in sql with grouping DEST_COUNTRY_NAME having count(1),stroing the data in sql_data variable
sql_data=spark.sql(""" select DEST_COUNTRY_NAME,count(1) from sql_csv_table where group by DEST_COUNTRY_NAME""")
sql_data.show()

+--------------------+--------+
|   DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
|            Anguilla|       1|
|              Russia|       1|
|            Paraguay|       1|
|             Senegal|       1|
|              Sweden|       1|
|            Kiribati|       1|
|              Guyana|       1|
|         Philippines|       1|
|            Djibouti|       1|
|            Malaysia|       1|
|           Singapore|       1|
|                Fiji|       1|
|              Turkey|       1|
|                Iraq|       1|
|             Germany|       1|
|              Jordan|       1|
|               Palau|       1|
|Turks and Caicos ...|       1|
|              France|       1|
|              Greece|       1|
+--------------------+--------+
only showing top 20 rows



In [10]:
#load the csv file and store it in csv_load variable and create a temporary table "sql_csv_table",
# write a query in sql with grouping DEST_COUNTRY_NAME having count(1),stroing the data in sql_data variable
#==========
#load the csv file and store it in csv_load variable, select the column DEST_COUNTRY_NAME having count from aesc store in sql_with_python variable
#explain the both
csv_load=spark.read.option("header","true").option("inferSchema","true").csv("2015-summary.csv")
sql_data=spark.sql(""" select DEST_COUNTRY_NAME,count(1) as count from sql_csv_table group by DEST_COUNTRY_NAME""")
sql_data.tail(5)
sql_with_python=csv_load.groupBy("DEST_COUNTRY_NAME").count()
sql_with_python.take(5)
sql_data.explain()
sql_with_python.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#114], functions=[count(1)])
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 0
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#114, 200), ENSURE_REQUIREMENTS, [plan_id=227]
            +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#114], functions=[partial_count(1)])
               +- FileScan csv [DEST_COUNTRY_NAME#114] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
+- == Initial Plan ==
   HashAggregate(keys=[DEST_COUNTRY_NAME#114], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#114, 200), ENSURE_REQUIREMENTS, [plan_id=217]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#114], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#114] Batched: false,

In [11]:
#load the csv file and store it in csv_load variable and create a temporary table "sql_csv_table",
# write a query in sql with max count column,stroing the data in sql_data variable
#==========
#load the csv file and store it in csv_load variable, select max of the count column store the result in pyspark_sql variable
from pyspark.sql.functions import max
sql_data=spark.sql(""" select max(count) from sql_csv_table """)
sql_data.show()
pyspark_sql=csv_load.select(max("count"))
pyspark_sql.show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [12]:
#load the csv file and store it in csv_load variable and create a temporary table "sql_csv_table",
#write a query in sql selecting DEST_COUNTRY_NAME and sum of count of grouping DEST_COUNTRY_NAME as destination,show the result in desc
#store the result in csv_sql variable
csv_load=spark.read.option("header","true").option("inferSchema","true").csv("2015-summary.csv")
csv_load.createOrReplaceTempView("sql_csv_table")
csv_sql=spark.sql(""" select DEST_COUNTRY_NAME, sum(count) as destination from sql_csv_table group by DEST_COUNTRY_NAME order by destination desc""")
csv_sql.show()

+------------------+-----------+
| DEST_COUNTRY_NAME|destination|
+------------------+-----------+
|     United States|     411352|
|            Canada|       8399|
|            Mexico|       7140|
|    United Kingdom|       2025|
|             Japan|       1548|
|           Germany|       1468|
|Dominican Republic|       1353|
|       South Korea|       1048|
|       The Bahamas|        955|
|            France|        935|
|          Colombia|        873|
|            Brazil|        853|
|       Netherlands|        776|
|             China|        772|
|           Jamaica|        666|
|        Costa Rica|        588|
|       El Salvador|        561|
|            Panama|        510|
|              Cuba|        466|
|             Spain|        420|
+------------------+-----------+
only showing top 20 rows



In [13]:
#load the csv file and store it in csv_load variable
#selecting DEST_COUNTRY_NAME and sum of count of grouping DEST_COUNTRY_NAME as destination,show the result in desc
from pyspark.sql.functions import desc
csv_load=spark.read.option("header","true").option("inferSchema","true").csv("2015-summary.csv")
csv_pyspark=csv_load.groupBy("DEST_COUNTRY_NAME") \
.sum("count") \
.withColumnRenamed("sum(count)","destination") \
.sort(desc("destination"))
csv_pyspark.show()

+------------------+-----------+
| DEST_COUNTRY_NAME|destination|
+------------------+-----------+
|     United States|     411352|
|            Canada|       8399|
|            Mexico|       7140|
|    United Kingdom|       2025|
|             Japan|       1548|
|           Germany|       1468|
|Dominican Republic|       1353|
|       South Korea|       1048|
|       The Bahamas|        955|
|            France|        935|
|          Colombia|        873|
|            Brazil|        853|
|       Netherlands|        776|
|             China|        772|
|           Jamaica|        666|
|        Costa Rica|        588|
|       El Salvador|        561|
|            Panama|        510|
|              Cuba|        466|
|             Spain|        420|
+------------------+-----------+
only showing top 20 rows



In [14]:
#load the by_day folder and store in load_byday_csv and create the sql temp table as retail_data, assign the schema of load_byday_csv to schema
#and print the schema
load_byday_csv=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day")
load_byday_csv.createOrReplaceTempView("retail_data")
schema=load_byday_csv.schema
print(schema)

StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])


In [15]:
#load the by_day folder and store in load_byday_csv and select CustomerID,UnitPrice * Quantity as total_cost,InvoiceDate
#group by CustomerID with window function as 1 day, sum the total and rename it to total_cost and sort with desc of total_cost
from pyspark.sql.functions import col,window,desc
load_by_data=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
check_total=load_by_data.selectExpr("CustomerID","(UnitPrice * Quantity) as total_cost","InvoiceDate") \
.groupBy(col("CustomerID"),window(col("InvoiceDate"),"1 day")) \
.sum("total_cost") \
.withColumnRenamed("sum(total_cost)","total_cost") \
.sort(desc("total_cost"))
check_total.show()

+----------+--------------------+------------------+
|CustomerID|              window|        total_cost|
+----------+--------------------+------------------+
|   17450.0|{2011-09-20 00:00...|          71601.44|
|      NULL|{2011-11-14 00:00...|          55316.08|
|      NULL|{2011-11-07 00:00...|          42939.17|
|      NULL|{2011-03-29 00:00...| 33521.39999999998|
|      NULL|{2011-12-08 00:00...|31975.590000000007|
|   18102.0|{2011-09-15 00:00...|31661.540000000005|
|      NULL|{2010-12-21 00:00...|31347.479999999938|
|   18102.0|{2011-10-21 00:00...|          29693.82|
|   18102.0|{2010-12-07 00:00...|          25920.37|
|   14646.0|{2011-10-20 00:00...|25833.559999999994|
|      NULL|{2010-12-10 00:00...|25399.560000000012|
|      NULL|{2010-12-17 00:00...|25371.769999999768|
|      NULL|{2011-11-25 00:00...|24148.069999999992|
|      NULL|{2011-11-29 00:00...|23744.250000000055|
|   12415.0|{2011-06-15 00:00...| 23426.81000000001|
|      NULL|{2010-12-06 00:00...|23395.0999999

In [16]:
#load the csv files and store in load_by_data variable, store the schema in schema variable
#read the stream by setting maxFilesPerTrigger as 1 and check if its streaming or not
load_by_data=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
schema=load_by_data.schema
print(schema)
read_stream=spark.readStream.schema(schema).option("maxFilesPerTrigger",1).option("header","true").format("csv").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
read_stream.isStreaming

StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])


True

In [17]:
#load the csv files and store in load_by_data variable, store the schema in schema variable
#read the stream by setting maxFilesPerTrigger as 1 and check if its streaming or not in read_stream variable
#with read_stream select CustomerID,UnitPrice * Quantity as total_cost,InvoiceDate
#group by CustomerID with window function as 1 day, sum the total_cost store the entire result in purchasebycustomerperhour
from pyspark.sql.functions import col,window
load_by_data=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
schema=load_by_data.schema
read_stream=spark.readStream.schema(schema).option("maxFilesPerTrigger",1).option("header","true").format("csv").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
purchasebycustomerperhour=read_stream.selectExpr("CustomerID","Quantity*UnitPrice as total_cost","InvoiceDate") \
.groupBy("CustomerID",window(col("InvoiceDate"),"1 day")) \
.sum("total_cost")


In [18]:
#stop the write_stream and with purchasebycustomerperhour write the stream,store in memory,and tablename is customer_purchases, output the mode as complete and start
write_stream=purchasebycustomerperhour.writeStream.format("memory").queryName("customer_purchases").outputMode("complete").start()

In [19]:
#select all the columns with sql in customer_purchases table and store in customer variable
customer=spark.sql(""" select * from customer_purchases""")
customer.show()

+----------+------+---------------+
|CustomerID|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



In [20]:
#load the csv files and store in load_by_data variable, store the schema in schema variable
#read the stream by setting maxFilesPerTrigger as 1 and check if its streaming or not in read_stream variable
#with read_stream select CustomerID,UnitPrice * Quantity as total_cost,InvoiceDate
#group by CustomerID with window function as 1 day, sum the total_cost store the entire result in purchasebycustomerperhour
#stop the write_stream and with purchasebycustomerperhour write the stream,store in console,and tablename is secondtb, output the mode as complete and start

from pyspark.sql.functions import col,window
load_by_data=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
schema=load_by_data.schema
read_stream=spark.readStream.schema(schema).option("maxFilesPerTrigger",1).option("header","true").format("csv").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
read_stream.isStreaming
purchasebycustomerperhour=read_stream.selectExpr("CustomerID","(UnitPrice*Quantity)as total_cost","InvoiceDate") \
.groupBy("CustomerID",window(col("InvoiceDate"),"1 day")) \
.sum("total_cost")
purchasebycustomerperhour.writeStream.format("console").queryName("secondtb").outputMode("complete").start()

In [21]:
staticDataFrame=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Spark_files/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")
staticSchema=staticDataFrame.schema

In [22]:
from pyspark.sql.functions import date_format,col
some_df=staticDataFrame \
.na.fill(0) \
.withColumn("day_of_week",date_format(col("InvoiceDate"),"EEEE")) \
.coalesce(5)

In [23]:
trainDataFrame=some_df.where("InvoiceDate < '2011-11-07'")
testDataFrame=some_df.where("InvoiceDate >= '2011-11-07'")
trainDataFrame.count()
#testDataFrame.count()

444958

In [24]:
from pyspark.sql import Row
rdd_df=spark.sparkContext.parallelize([Row(value=1),Row(value=2),Row(value=3),Row(value=4)]).toDF()
rdd_df.show()


+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
+-----+



In [25]:
#create a dataframe number_500 contains 500 numbers and add each value + 10 and show the result
number_500=spark.range(500).toDF("numbers")
number_500.select(number_500["numbers"]+10).show()

+--------------+
|(numbers + 10)|
+--------------+
|            10|
|            11|
|            12|
|            13|
|            14|
|            15|
|            16|
|            17|
|            18|
|            19|
|            20|
|            21|
|            22|
|            23|
|            24|
|            25|
|            26|
|            27|
|            28|
|            29|
+--------------+
only showing top 20 rows



In [26]:
#range of 2 numbers collect the numbers and store in rows_collect and print rows_collect
rows_collect=spark.range(2).collect()
print(rows_collect)

[Row(id=0), Row(id=1)]


In [27]:
from pyspark.sql.types import *
b=ByteType()
print(b)

ByteType()


In [28]:
#load the json data and store in json_data variable and show the result
json_data=spark.read.format("json").load("2015-summary.json")
json_data.show()
json_data.schema
#json_data.printSchema()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [29]:
#manually assign schema of to the json_data by loading and select only count column and store the result in a 
from pyspark.sql.types import StructField,StructType,LongType,StringType
from pyspark.sql.functions import col
schema=StructType([StructField("DEST_COUNTRY_NAME",StringType(),True),StructField("ORIGIN_COUNTRY_NAME",StringType(),True),StructField("count",LongType(),True)])
json_data=spark.read.format("json").schema(schema).load("2015-summary.json")
json_data.show()
a= json_data.select(col("count"))
a.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [30]:
#access only columns of json data and store in json_data and print the result
json_data=spark.read.format("json").load("2015-summary.json").columns
print(json_data)

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']


In [31]:
#load the json data in json_data variable and print first record as a row type
json_data=spark.read.format("json").load("2015-summary.json")
json_data.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [32]:
#create a row of values manually and show the result of 2nd index
from pyspark.sql import Row
row_data=Row("1",None,1,False)
print(row_data[2])

1


In [33]:
#load the json data into json_data variable and create temp table df_table
json_data=spark.read.format("json").load("2015-summary.json")
json_data.createOrReplaceTempView("df_table")

In [34]:
#in manual_schema store schema(string,string,int) and df_data store random data 
#and create the dataframe in new_df variable with df_data and manual_schema and print the result of new_df
from pyspark.sql.types import StructField,StructType,StringType,LongType
from pyspark.sql import Row
manual_schema=StructType([StructField("first",StringType(),True),StructField("second",StringType(),True),StructField("third",LongType(),True)])
df_data=[Row("hi","sushmitha",1),Row("hello","dolly",2)]
new_df=spark.createDataFrame(df_data,manual_schema)
new_df.show()

+-----+---------+-----+
|first|   second|third|
+-----+---------+-----+
|   hi|sushmitha|    1|
|hello|    dolly|    2|
+-----+---------+-----+



In [35]:
#load the json data in json_data variable and select only "DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME" and show the result
json_data=spark.read.format("json").load("2015-summary.json")
json_data.show()
json_data.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME").show(2)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [36]:
#load the json data in json_data variable and using col,column,expr select the column DEST_COUNTRY_NAME and show the result of 2 records
from pyspark.sql.functions import expr,col,column
json_data=spark.read.format("json").load("2015-summary.json")
json_data.select(col("DEST_COUNTRY_NAME"),expr("DEST_COUNTRY_NAME"),column("DEST_COUNTRY_NAME")).show(2)
json_data.select(col("DEST_COUNTRY_NAME"),"DEST_COUNTRY_NAME").show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows

+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+
|    United States|    United States|
|    United States|    United States|
+-----------------+-----------------+
only showing top 2 rows



In [37]:
#load the json data in json_data variable and select the column DEST_COUNTRY_NAME as destination with expr and show 2 records
from pyspark.sql.functions import expr
json_data=spark.read.format("json").load("2015-summary.json")
json_data.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [38]:
#load the json data in json_data variable and select the column DEST_COUNTRY_NAME as dest with expr, again rename to destination1 and show 5 records
from pyspark.sql.functions import expr
json_data=spark.read.format("json").load("2015-summary.json")
json_data.select(expr("DEST_COUNTRY_NAME AS dest").alias("destination1")).show(5)

+-------------+
| destination1|
+-------------+
|United States|
|United States|
|United States|
|        Egypt|
|United States|
+-------------+
only showing top 5 rows



In [39]:
#load the json data in json_data variable and select the column DEST_COUNTRY_NAME as dest, DEST_COUNTRY_NAME with selectExpr, show 2 records
json_data=spark.read.format("json").load("2015-summary.json")
json_data.selectExpr("DEST_COUNTRY_NAME as dest","DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|         dest|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [40]:
#load the json data in json_data variable and select all columns, if DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME store in withincountry and show 5 records
json_data=spark.read.format("json").load("2015-summary.json")
json_data.selectExpr("*","(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withincountry").show(5)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withincountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
|            Egypt|      United States|   15|        false|
|    United States|              India|   62|        false|
+-----------------+-------------------+-----+-------------+
only showing top 5 rows



In [41]:
#load the json data in json_data variable and select avg of count,count of distinct DEST_COUNTRY_NAME
json_data=spark.read.format("json").load("2015-summary.json")
json_data.selectExpr("avg(count)","count(distinct(DEST_COUNTRY_NAME))").show()

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [42]:
#select all columns of json_data and add new column as One stroing lit value of 1 and print 2 records
from pyspark.sql.functions import lit
json_data.select(expr("*"),lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [43]:
#load the json data in json_data variable
#a variable contains data of adding new column name as new_column contains value of literal 1 and show 2 records
#b variable contains data of adding new column name as new_column contains value if DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME and show 2 records
#c variable contains data of adding new column name as destination contains value of DEST_COUNTRY_NAME and show 2 records
#d variable contains data of renaming column DEST_COUNTRY_NAME to new_column destination and show 2 records
json_data=spark.read.format("json").load("2015-summary.json")
a=json_data.withColumn("new_column",lit(1))
a.show(2)
b=json_data.withColumn("new_column",expr("DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME"))
b.show(2)
c=json_data.withColumn("destination",expr("DEST_COUNTRY_NAME"))
c.show(2)
c.columns
d=json_data.withColumnRenamed("DEST_COUNTRY_NAME","destination")
d.show(2)

+-----------------+-------------------+-----+----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|new_column|
+-----------------+-------------------+-----+----------+
|    United States|            Romania|   15|         1|
|    United States|            Croatia|    1|         1|
+-----------------+-------------------+-----+----------+
only showing top 2 rows

+-----------------+-------------------+-----+----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|new_column|
+-----------------+-------------------+-----+----------+
|    United States|            Romania|   15|     false|
|    United States|            Croatia|    1|     false|
+-----------------+-------------------+-----+----------+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|  destination|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|United States|
|    United States|       

In [44]:
#load the json data in json_data variable
#a variable contains data of adding new column name as "This Long Column-name" contains value of ORIGIN_COUNTRY_NAME and show 2 records
#b variable contains data of selecting new column name as "This Long Column-name" and selecting "This Long Column-name" as new_column and show 2 records
#c variable contains data of selecting new column name as "This Long Column-name" and columns and print the result
from pyspark.sql.functions import expr
json_data=spark.read.format("json").load("2015-summary.json")
a=json_data.withColumn("This Long Column-name",expr("ORIGIN_COUNTRY_NAME"))
a.show(2)
b=a.selectExpr("`This Long Column-name`","`This Long Column-name` as new_column")
b.show(2)
c=a.select(expr("`This Long Column-name`")).columns
print(c)

+-----------------+-------------------+-----+---------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|This Long Column-name|
+-----------------+-------------------+-----+---------------------+
|    United States|            Romania|   15|              Romania|
|    United States|            Croatia|    1|              Croatia|
+-----------------+-------------------+-----+---------------------+
only showing top 2 rows

+---------------------+----------+
|This Long Column-name|new_column|
+---------------------+----------+
|              Romania|   Romania|
|              Croatia|   Croatia|
+---------------------+----------+
only showing top 2 rows

['This Long Column-name']


In [45]:
#load the json data in json_data variable and delete the column DEST_COUNTRY_NAME and show 2 records
json_data=spark.read.format("json").load("2015-summary.json")
json_data.drop("DEST_COUNTRY_NAME").show(2)

+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|            Romania|   15|
|            Croatia|    1|
+-------------------+-----+
only showing top 2 rows



In [46]:
#load the json data in json_data variable and 
#a variable contains data of adding new column name as "new_column" contains value of "count" column and show 2 records,schema
#b variable contains data of a and selecting new_column column name and change the type to int and show the schema

json_data=spark.read.format("json").load("2015-summary.json")
a=json_data.withColumn("new_column",expr("count"))
print(a.schema)
a.show(2)
b=a.withColumn("new_column",col("new_column").cast("int"))
print(b.schema)

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True), StructField('new_column', LongType(), True)])
+-----------------+-------------------+-----+----------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|new_column|
+-----------------+-------------------+-----+----------+
|    United States|            Romania|   15|        15|
|    United States|            Croatia|    1|         1|
+-----------------+-------------------+-----+----------+
only showing top 2 rows

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True), StructField('new_column', IntegerType(), True)])


In [47]:
#load the json data in json_data variable and 
#a variable contains data of count<2 with where condition and show 4 records
#b variable contains data of count<2 with filter condition and show 4 records
#c variable contains data of count<2,ORIGIN_COUNTRY_NAME!=Croatia with where condition and show 4 records
#d variable contains data of "DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME" distinct and show count of records
#e variable contains data of "DEST_COUNTRY_NAME" distinct and show count of records
from pyspark.sql.functions import col
json_data=spark.read.format("json").load("2015-summary.json")
a=json_data.where("count<2")
a.show(4)
b=json_data.filter(col("count")<2)
b.show(4)
c=json_data.where("count<2").where(col("ORIGIN_COUNTRY_NAME")!="Croatia")
c.show(4)
d=json_data.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME").distinct().count()
print(d)
e=json_data.select("ORIGIN_COUNTRY_NAME").distinct().count()
print(e)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
+-----------------+-------------------+-----+
only showing top 4 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|          Moldova|      United States|    1|
|            Malta|      United States|    1|
+-----------------+-------------------+-----+
only showing top 4 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|          Singapore|    1|
|          Moldova|      Unite

In [48]:
#load the json data in json_data variable, with sample split the data into 50% with constant 5 with replacement as false and count the result
json_data=spark.read.format("json").load("2015-summary.json")
seed=5
withReplacement=False
fraction=0.5
json_data.sample(withReplacement,fraction,seed).count()

138

In [49]:
#import the json data in json_data variable and split the data into 25%,75% with seed 5 and check count if its true or false
json_data=spark.read.format("json").load("2015-summary.json")
seed=5
random_split=json_data.randomSplit([0.25,0.75],seed)
random_split[0].count()>random_split[1].count()

False

In [50]:
#load the json data into json_data variable and store the schema of json_data in schema variable
#add new_rows contains ("sush","asara",2),("dolly","asara",4)
#parallelize the new_rows store in parallelize variable
#create a dataframe with parallelize ,schema in new_df variable and show the result of new_df
#json_data union new_df where count=2,dest_country_name!=origin_country_name and show the result
from pyspark.sql import Row
json_data=spark.read.format("json").load("2015-summary.json")
schema=json_data.schema
print(schema)
new_rows=[Row("sush","asara",2),Row("dolly","asara",4)]
parallelize=spark.sparkContext.parallelize(new_rows)
print(parallelize)
new_df=spark.createDataFrame(parallelize,schema)
new_df.show()
json_data.union(new_df).where(col("count")==2).where(col("DEST_COUNTRY_NAME")!=col("ORIGIN_COUNTRY_NAME")).show()

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])
ParallelCollectionRDD[1236] at readRDDFromFile at PythonRDD.scala:289
+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|             sush|              asara|    2|
|            dolly|              asara|    4|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Liberia|      United States|    2|
|          Hungary|      United States|    2|
|    United States|            Vietnam|    2|
|         Malaysia|      United States|    2|
|          Croatia|      United States|    2|
|    United States|            Liberia|    2|
|    United States|              Malta|    2|
|          Georgia|      Unite

In [51]:
#load the json data in json_data variable
#in a variable sort the result of count with sort and show 5 records
#in b variable sort the result of count,DEST_COUNTRY_NAME with orderBy and show 5 records
json_data=spark.read.format("json").load("2015-summary.json")
a=json_data.sort("count")
a.show(5)
b=json_data.orderBy(col("count"),col("DEST_COUNTRY_NAME"))
b.show(5)
c=json_data.orderBy("count","DEST_COUNTRY_NAME")
c.show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

In [52]:
#load the json data in json_data variable
#sort the count values in desc with orderby in a variable and show 2 records
#sort the count values in desc and dest_country_name in asc with orderby in b variable and show 2 records
from pyspark.sql.functions import asc,desc,col
json_data=spark.read.format("json").load("2015-summary.json")
a=json_data.orderBy(expr("count desc"))
a.show(2)
b=json_data.orderBy(col("DEST_COUNTRY_NAME").asc(),col("count").desc())
b.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Algeria|      United States|    4|
|           Angola|      United States|   15|
+-----------------+-------------------+-----+
only showing top 2 rows



In [53]:
#load the json data in json_data variable and using limit with 5 show the result
#in a variable sort the count desc and using limit with 5 show the result
#in b variable repartition json_data to 5 and check how many partitioned it got
#in c variable repartition json_data based on column ORIGIN_COUNTRY_NAME and check how many partitioned it got
#in d variable repartition json_data based on column DEST_COUNTRY_NAME with 4 and check how many partitioned it got
from pyspark.sql.functions import desc,col
json_data=spark.read.format("json").load("2015-summary.json")
json_data.limit(5).show()
a=json_data.orderBy(col("count").desc()).limit(5)
a.show()
json_data.rdd.getNumPartitions()
b=json_data.repartition(5)
b.rdd.getNumPartitions()
c=json_data.repartition(col("ORIGIN_COUNTRY_NAME"))
c.rdd.getNumPartitions()
d=json_data.repartition(4,col("DEST_COUNTRY_NAME"))
d.rdd.getNumPartitions()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
|           Canada|      United States|  8399|
|    United States|             Mexico|  7187|
|           Mexico|      United States|  7140|
+-----------------+-------------------+------+



4

In [54]:
#load the json data in json_data variable and print the result with show,take,show(value,False),collect
json_data=spark.read.format("json").load("2015-summary.json")
#json_data.take(5)
json_data.show(5)
#json_data.collect()
json_data.show(10,False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
|United States    |Singapore          |1    |
|United States    |Grenada            |62   |
|Costa Rica       |United States      |588  |
|Senegal          |United States      |40   |
|Moldova 

In [55]:
#load the csv retail data in retail_data variable and print the result with show and 
#create a temp table retail_temp_table and print schema of retail_data
retail_data=spark.read.option("inferSchema","true").option("header","true").csv("2010-12-01.csv")
retail_data.show(5,False)
retail_data.createOrReplaceTempView("retail_temp_table")
retail_data.printSchema()

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [56]:
#import lit and select the 5,0.5,five
from pyspark.sql.functions import lit
retail_data.select(lit(5),lit("five"),lit(0.5))

DataFrame[5: int, five: string, 0.5: double]

In [57]:
# from retail_data select invoiceno,description where InvoiceNo not equal to 536365 and show the result
retail_data.where(col("InvoiceNo")!="536365") \
.select("InvoiceNo","Description") \
.show(5,False)


+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [58]:
#load the csv retail data in retail_data variable and print the result with show and 
#in pricefilter variable check UnitPrice is grater tan 600
#in descriptionfilter variable check Description col contains POSTAGE and greater than or equal to 1 with instr
#with reatial_data check StockCode is dot with == and pricefilter or descriptionfilter and show
#with reatial_data check StockCode is dot with isin and pricefilter or descriptionfilter and show
from pyspark.sql.functions import instr
retail_data=spark.read.format("csv").option("header","true").option("inferSchema","true").load("2010-12-01.csv")
pricefilter=col("UnitPrice") > 600
descriptionfilter=instr(col("Description"),"POSTAGE") >= 1
retail_data.where(col("StockCode") == "DOT").where(pricefilter | descriptionfilter).show(5,False)

retail_data.where(retail_data.StockCode.isin("DOT")).where(pricefilter | descriptionfilter).show(5,False)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|536544   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 14:32:00|569.77   |NULL      |United Kingdom|
|536592   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 17:06:00|607.49   |NULL      |United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|536544   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 14:32:00|569.77   |NULL  

In [59]:
#load the csv retail data in retail_data variable and print the result with show and 
#in pricefilter variable check UnitPrice is grater tan 60
#in stockcodefilter variable check stockcodefilter contains dot init
#in descriptionfilter variable check Description col contains POSTAGE and greater than or equal to 1 with instr
#with reatial_data add new column isexpensive where  descriptionfilter and (pricefilter or stockcodefilter) in checking_filters variable
#and show if isexpensive is true
from pyspark.sql.functions import instr
retail_data=spark.read.format("csv").option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
retail_data.show(5,False)
descriptionfilter=instr(col("Description"),"POSTAGE")>=1
pricefilter=col("UnitPrice")>600
stockcodefilter=col("StockCode").isin("DOT")
checking_filters=retail_data.withColumn("isexpensive",descriptionfilter & (pricefilter | stockcodefilter))
checking_filters.where("isexpensive").show(5,False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [60]:
#load the csv retail data in retail_data variable and print the result with show and 
#add new column isexpensive where not(unitprice) is less than or equal to 250 and show if isexpensive is true
retail_data=spark.read.format("csv").option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
retail_data.show(5,False)
retail_data.withColumn("isexpensive",expr("NOT UnitPrice<=250")).where("isexpensive").show()
retail_data.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [61]:
#load the csv retail data in retail_data variable and check col description value contains hello with eqnullsafe
retail_data=spark.read.option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
retail_data.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [62]:
#load the csv retail data in retail_data variable and 
#using pow function ((Quantity*UnitPrice) ^2 )+5 store in fabricated_quantity variable
#and show the result with just select statement of 2 records of columns CustomerID and its fabricated_quantity as "actual"
#and show the result with just selectexpr statement of 2 records of columns CustomerID and its fabricated_quantity as "actual"

from pyspark.sql.functions import expr,pow
retail_data=spark.read.option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
fabricated_quantity=pow(col("Quantity")*col("UnitPrice"),2)+5
retail_data.select(col("CustomerID"),fabricated_quantity.alias("actual")).show(2)
retail_data.selectExpr("CustomerID","POWER((Quantity * UnitPrice),2)+5 as actual").show(2)

+----------+------------------+
|CustomerID|            actual|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows

+----------+------------------+
|CustomerID|            actual|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [63]:
#load the csv retail data in retail_data variable and 
#with round(lit(2.5)),bround(lit(2.5)) check what does the output
from pyspark.sql.functions import lit,round,bround
retail_data=spark.read.option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
retail_data.select(round(lit(2.5)),bround(lit(2.5))).show(3)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 3 rows



In [64]:
#load the csv retail data in retail_data variable and 
#with corr between columns UnitPrice,Quantity check how much its linear relationship 
#with describe function check the retail_data of count,min,max,mean,stddev
from pyspark.sql.functions import corr
retail_data.select(corr(col("Quantity"),col("UnitPrice"))).show()
retail_data.select(corr(col("UnitPrice"),col("Quantity"))).show()
retail_data.describe().show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+

+-------------------------+
|corr(UnitPrice, Quantity)|
+-------------------------+
|     -0.04112314436835552|
+-------------------------+

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                NULL| 8.627413127413128| 4.151946589446603|15661.388719512195|          NULL|
| stddev|72.894478

In [65]:
#colname variable contains unitprice,quantileprobs variable contains list of 0.5 value, relerror contains 0.05 value
#check .stat.approxQuantile with 3 varibales with retail_data
#check .stat.crosstab with "StockCode","Quantity" varibales with retail_data
#check .stat.freqItems with "StockCode","Quantity" varibales with retail_data
#with monotonically_increasing_id select and show 2 records
from pyspark.sql.functions import monotonically_increasing_id
colName="UnitPrice"
quantileProbs=[0.5,0.25]
relError=0.05
retail_data.stat.approxQuantile(colName,quantileProbs,relError)
retail_data.stat.crosstab("StockCode","Quantity").show(5)
retail_data.stat.freqItems(["StockCode","Quantity"]).show(5)
retail_data.select(monotonically_increasing_id()).show(2)

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             21259|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  2|  0|  0|  0|  0| 

In [66]:
#load the csv retail data in retail_data variable and 
#use function initcap,upper,lower for description variable and show result
from pyspark.sql.functions import initcap,lower,upper
retail_data=spark.read.option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
retail_data.select(initcap(col("Description"))).show(5,False)
retail_data.select(lower("Description"),upper(col("Description"))).show(5,False)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
|Red Woolly Hottie White Heart.     |
+-----------------------------------+
only showing top 5 rows

+-----------------------------------+-----------------------------------+
|lower(Description)                 |upper(Description)                 |
+-----------------------------------+-----------------------------------+
|white hanging heart t-light holder |WHITE HANGING HEART T-LIGHT HOLDER |
|white metal lantern                |WHITE METAL LANTERN                |
|cream cupid hearts coat hanger     |CREAM CUPID HEARTS COAT HANGER     |
|knitted union flag hot water bottle|KNITTED UNION FLAG HOT WATER BOTTLE|
|red woolly hottie white heart.     |RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+----

In [67]:
#using ltrim,rtrim,rpad,lpad,trim select ("    HELLO    ") and check the result
from pyspark.sql.functions import ltrim,rtrim,rpad,lpad,trim
retail_data.select(ltrim(lit("    HELLO    ")),rtrim(lit("    HELLO    ")),trim(lit("    HELLO    "))).show(1)
retail_data.select(lpad(lit(" pHELLO"),13,"2"),rpad(lit(" p HELLO "),10,"6")).show(1)

+--------------------+--------------------+-------------------+
|ltrim(    HELLO    )|rtrim(    HELLO    )|trim(    HELLO    )|
+--------------------+--------------------+-------------------+
|           HELLO    |               HELLO|              HELLO|
+--------------------+--------------------+-------------------+
only showing top 1 row

+--------------------+----------------------+
|lpad( pHELLO, 13, 2)|rpad( p HELLO , 10, 6)|
+--------------------+----------------------+
|       222222 pHELLO|             p HELLO 6|
+--------------------+----------------------+
only showing top 1 row



In [68]:
#using regexp_replace wherever col description contains BLACK|WHITE|RED|GREEN|BLUE then replaced with sush
#using translate wherever hanging comes replace with 1234
from pyspark.sql.functions import regexp_replace,translate
regex_string="BLACK|WHITE|RED|GREEN|BLUE"
retail_data.select(regexp_replace(col("Description"),regex_string,"sush")).show(5,False)
retail_data.select(translate(col("Description"),"HANGING","1234")).show(5,False)

+----------------------------------------------------------------+
|regexp_replace(Description, BLACK|WHITE|RED|GREEN|BLUE, sush, 1)|
+----------------------------------------------------------------+
|sush HANGING HEART T-LIGHT HOLDER                               |
|sush METAL LANTERN                                              |
|CREAM CUPID HEARTS COAT HANGER                                  |
|KNITTED UNION FLAG HOT WATER BOTTLE                             |
|sush WOOLLY HOTTIE sush HEART.                                  |
+----------------------------------------------------------------+
only showing top 5 rows

+-------------------------------------+
|translate(Description, HANGING, 1234)|
+-------------------------------------+
|W1TE 123434 1E2RT T-L41T 1OLDER      |
|W1TE MET2L L23TER3                   |
|CRE2M CUPD 1E2RTS CO2T 1234ER        |
|K3TTED U3O3 FL24 1OT W2TER BOTTLE    |
|RED WOOLLY 1OTTE W1TE 1E2RT.         |
+-------------------------------------+
only showing

In [69]:
#using regexp_extract wherever col description contains BLACK|WHITE|RED|GREEN|BLUE then extract the first
# For each row, it returns the extracted color (if any) along with the original "Description" value and then displays the first 5 result
from pyspark.sql.functions import regexp_extract
regexp_string="(BLACK|WHITE|RED|CREAM|BLUE)"
retail_data.select(regexp_extract(col("Description"),regexp_string,1),col("Description")).show(5)

+------------------------------------------------------------+--------------------+
|regexp_extract(Description, (BLACK|WHITE|RED|CREAM|BLUE), 1)|         Description|
+------------------------------------------------------------+--------------------+
|                                                       WHITE|WHITE HANGING HEA...|
|                                                       WHITE| WHITE METAL LANTERN|
|                                                       CREAM|CREAM CUPID HEART...|
|                                                            |KNITTED UNION FLA...|
|                                                         RED|RED WOOLLY HOTTIE...|
+------------------------------------------------------------+--------------------+
only showing top 5 rows



In [70]:
#contains_black variable check col(Description) contains black,contains_white variable check col(Description) contains white
#add new column to retail_data haswhiteorblack with or condition check it contains black or white and that is true and select only description
from pyspark.sql.functions import instr
retail_data=spark.read.option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
contains_black=instr(col("Description"),"BLACK")>=1
contains_white=instr(col("Description"),"WHITE")>=1
retail_data.withColumn("haswhiteorblack",contains_black | contains_white).where("haswhiteorblack").select(col("Description")).show(5,False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
+----------------------------------+
only showing top 5 rows



In [71]:
#create a list simplecolors=["black","white","red","blue","green"]
#selectedcolumns varibale is an empty list
#create a function colo_locator 
#you have a DataFrame named retail_data with a column Description containing text descriptions of retail items. 
#You want to create new boolean columns indicating the presence of specific colors in the Description column with is_red,is_black etc
#After that, you wish to filter and display the rows where the Description contains either "white" or "red".
from pyspark.sql.functions import locate
simplecolors=["black","white","red","blue","green"]
selectedcolumns=[]

def color_locator(column,color_string):
    return locate(color_string.upper(),column).cast("boolean").alias("is_"+color_string)
for i in simplecolors:
    a=color_locator(retail_data.Description,i)
    selectedcolumns.append(a)
print(selectedcolumns)
selectedcolumns.append(col("Description"))
retail_data.select(*selectedcolumns).where(expr("is_white or is_red")).show(3,False)

[Column<'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS is_black'>, Column<'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS is_white'>, Column<'CAST(locate(RED, Description, 1) AS BOOLEAN) AS is_red'>, Column<'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS is_blue'>, Column<'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS is_green'>]
+--------+--------+------+-------+--------+----------------------------------+
|is_black|is_white|is_red|is_blue|is_green|Description                       |
+--------+--------+------+-------+--------+----------------------------------+
|false   |true    |false |false  |false   |WHITE HANGING HEART T-LIGHT HOLDER|
|false   |true    |false |false  |false   |WHITE METAL LANTERN               |
|false   |true    |true  |false  |false   |RED WOOLLY HOTTIE WHITE HEART.    |
+--------+--------+------+-------+--------+----------------------------------+
only showing top 3 rows



In [72]:
#create a dataframe datadf of range(10) and adding two columns today column contains current_date and now column contains current_timestamp
#create a temp table "datatable" with datadf
#print the schema of datadf
from pyspark.sql.functions import current_date,current_timestamp
datadf=spark.range(10).withColumn("today",current_date()).withColumn("now",current_timestamp())
datadf.createOrReplaceTempView("datatable")
datadf.printSchema()

datadf.show(10,False)

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

+---+----------+--------------------------+
|id |today     |now                       |
+---+----------+--------------------------+
|0  |2025-04-01|2025-04-01 10:28:55.597576|
|1  |2025-04-01|2025-04-01 10:28:55.597576|
|2  |2025-04-01|2025-04-01 10:28:55.597576|
|3  |2025-04-01|2025-04-01 10:28:55.597576|
|4  |2025-04-01|2025-04-01 10:28:55.597576|
|5  |2025-04-01|2025-04-01 10:28:55.597576|
|6  |2025-04-01|2025-04-01 10:28:55.597576|
|7  |2025-04-01|2025-04-01 10:28:55.597576|
|8  |2025-04-01|2025-04-01 10:28:55.597576|
|9  |2025-04-01|2025-04-01 10:28:55.597576|
+---+----------+--------------------------+



In [73]:
#import date_add,date_sub and with existing datadf dataframe ad 5 days to today column and subtract 5 days to today column and show
from pyspark.sql.functions import date_add,date_sub 
datadf.select(date_add(col("today"),5),date_sub(col("today"),5)).show(5,False)

+------------------+------------------+
|date_add(today, 5)|date_sub(today, 5)|
+------------------+------------------+
|2025-04-06        |2025-03-27        |
|2025-04-06        |2025-03-27        |
|2025-04-06        |2025-03-27        |
|2025-04-06        |2025-03-27        |
|2025-04-06        |2025-03-27        |
+------------------+------------------+
only showing top 5 rows



In [74]:
#import 3 functions datediff,months_between,to_date
#with existing datadiff dataframe add new column week_age contains data of subtract 7 days of today column and checkdiff with datediff and show diff
#with existing datadiff dataframe lit("2016-01-01"),lit("2017-05-22") convert the 2 lit into date with to_date and start_date and end_date columns
#select months_between diff of 2 columns of start_date and end_date columns
from pyspark.sql.functions import datediff,months_between,to_date
datadf.withColumn("week_ago",date_sub(col("today"),7)).select(datediff(col("week_ago"),col("today"))).show(1)
datadf.select(to_date(lit("2016-01-01")).alias("start_date"),to_date(lit("2017-05-22")).alias("end_date")) \
.select(months_between(col("start_date"),col("end_date"))).show(5)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row

+------------------------------------------+
|months_between(start_date, end_date, true)|
+------------------------------------------+
|                              -16.67741935|
|                              -16.67741935|
|                              -16.67741935|
|                              -16.67741935|
|                              -16.67741935|
+------------------------------------------+
only showing top 5 rows



In [82]:
#create a dataframe with range(5) and add new column "date" contains value of "2000-10-16" and while select convert into str to date with to_date
#with existing datadf dataframe select "2000-10-16","2000-16-10" by converting into date with to_date
datadf1=spark.range(5).withColumn("date",lit("2000-10-16"))
datadf1.show(1)
datadf=spark.range(5).withColumn("date",lit("2000-10-16")).select(to_date(col("date")))
datadf.show(1)
datadf.select(to_date(lit("2000-16-10")),to_date(lit("2000-10-16"))).show(1)

+---+----------+
| id|      date|
+---+----------+
|  0|2000-10-16|
+---+----------+
only showing top 1 row

+-------------+
|to_date(date)|
+-------------+
|   2000-10-16|
+-------------+
only showing top 1 row

+-------------------+-------------------+
|to_date(2000-16-10)|to_date(2000-10-16)|
+-------------------+-------------------+
|               NULL|         2000-10-16|
+-------------------+-------------------+
only showing top 1 row



In [94]:
#import to_timestamp function 
#in dateFormat variable store the format 'yyyy-dd-MM'
#create dataframe range(1) and
#selectint 2 columns with to_date 1 is "first" contains value "2000-16-10" and "second" contains value "2000-20-12"
#with to_timestamp select first and second columns
from pyspark.sql.functions import to_timestamp
dateFormat='yyyy-dd-MM'
cleandatediff=spark.range(1).select(to_date(lit("2000-16-10"),dateFormat).alias("first"),to_date(lit("2000-20-12"),dateFormat).alias("second"))
cleandatediff.show()

cleandatediff.select(to_timestamp(col("first")),to_timestamp(col("second"))).show(1)
cleandatediff.filter(col("first")<col("second")).show()

+----------+----------+
|     first|    second|
+----------+----------+
|2000-10-16|2000-12-20|
+----------+----------+

+-------------------+--------------------+
|to_timestamp(first)|to_timestamp(second)|
+-------------------+--------------------+
|2000-10-16 00:00:00| 2000-12-20 00:00:00|
+-------------------+--------------------+

+----------+----------+
|     first|    second|
+----------+----------+
|2000-10-16|2000-12-20|
+----------+----------+



In [106]:
#import struct, using struct select "Description","InvoiceNo" columns as "complex"
#create a table complexDF_table from the complexDF dataframe
from pyspark.sql.functions import struct
retail_data=spark.read.option("header","true").option("inferSchema","true").csv("2010-12-01.csv")
retail_data.show(1)
#retail_data.selectExpr("(Description,InvoiceNo) as complex","*").show(1,False)
complexDF=retail_data.select(struct("Description","InvoiceNo").alias("complex"),"*")
complexDF.show(1,False)
complexDF.createOrReplaceTempView("complexDF_table")

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 1 row

+--------------------------------------------+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|complex                                     |InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+--------------------------------------------+---------+---------+----------------------------------+-

In [115]:
#from complexDF dataframe select col complex,col complex of description,col complex of InvoiceNo with getfield
#select complex data in different columns like description and invoiceno
complexDF.select("complex").show(1,False)
complexDF.select("complex.Description").show(1,False)
complexDF.select(col("complex").getField("InvoiceNo")).show(1,False)
complexDF.select(col("complex.*")).show(1,False)

+--------------------------------------------+
|complex                                     |
+--------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER, 536365}|
+--------------------------------------------+
only showing top 1 row

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
+----------------------------------+
only showing top 1 row

+-----------------+
|complex.InvoiceNo|
+-----------------+
|536365           |
+-----------------+
only showing top 1 row

+----------------------------------+---------+
|Description                       |InvoiceNo|
+----------------------------------+---------+
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |
+----------------------------------+---------+
only showing top 1 row



In [124]:
#import the spli function, with retail_data select description and 
#for every spac split the description and rename new column as array_col
from pyspark.sql.functions import split
retail_data.select(col("Description"),split(col("Description")," ")).show(1,False)
retail_data.select(col("Description"),split(col("Description")," ").alias("array_col")).show(1,False)

+----------------------------------+----------------------------------------+
|Description                       |split(Description,  , -1)               |
+----------------------------------+----------------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
+----------------------------------+----------------------------------------+
only showing top 1 row

+----------------------------------+----------------------------------------+
|Description                       |array_col                               |
+----------------------------------+----------------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
+----------------------------------+----------------------------------------+
only showing top 1 row



In [131]:
#import the size function, with retail_data select description and 
#check the size after spliting the col description with " " 
from pyspark.sql.functions import size
retail_data.select(col("description"),size(split(col("description")," "))).show(2,False)

+----------------------------------+-------------------------------+
|description                       |size(split(description,  , -1))|
+----------------------------------+-------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|5                              |
|WHITE METAL LANTERN               |3                              |
+----------------------------------+-------------------------------+
only showing top 2 rows



In [137]:
#import the array_contains function, with retail_data select description and 
#check the array_contains has white in description column
from pyspark.sql.functions import array_contains
retail_data.select(array_contains(split(col("Description")," "),"WHITE")).show(1,False)

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|true                                            |
+------------------------------------------------+
only showing top 1 row



In [141]:
#import the explode function, with retail_data add new column "splitted" contains splitting of description with " "
#and add another new column "exploded" contains explod of splitted column
#and select columns "Description", "InvoiceNo", "exploded"
from pyspark.sql.functions import explode
retail_data.withColumn("splitted",split(col("Description")," ")) \
.withColumn("exploded",explode(col("splitted"))) \
.select("Description", "InvoiceNo", "exploded").show(2,False)

+----------------------------------+---------+--------+
|Description                       |InvoiceNo|exploded|
+----------------------------------+---------+--------+
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |WHITE   |
|WHITE HANGING HEART T-LIGHT HOLDER|536365   |HANGING |
+----------------------------------+---------+--------+
only showing top 2 rows



In [147]:
#import the create_map function, with retail_data select create_map of Description and InvoiceNo
#rename it as complex_map
#do the same as above but now extra select like new_col[some value]
from pyspark.sql.functions import create_map
retail_data.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map")).show(2,False)
retail_data.select(create_map(col("Description"),col("InvoiceNo")).alias("complex_map")) \
.selectExpr("complex_map['WHITE METAL LANTERN']").show()

+----------------------------------------------+
|complex_map                                   |
+----------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER -> 536365}|
|{WHITE METAL LANTERN -> 536365}               |
+----------------------------------------------+
only showing top 2 rows

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            NULL|
|                          536365|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|                            NULL|
|               

In [153]:
udfexampledf=spark.range(5).toDF("num")
def power3(double_value):
    return double_value**3
power3(3)

27

In [154]:
from pyspark.sql.functions import udf
power3udf=udf(power3)